<a href="https://colab.research.google.com/github/kirillkobychev/MLHSE/blob/homework/Gradien_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 4. Градиентный спуск

Задание выполнил(а):

    (Кобычев Кирилл)

## Общая информация

__Внимание!__  

Домашнее задание выполняется самостоятельно. Не допускается помощь в решении домашнего задания от однокурсников или третьих лиц. «Похожие» решения считаются плагиатом, и все задействованные студенты — в том числе и те, у кого списали, — не могут получить за него больше 0 баллов. Использование в решении домашнего задания генеративных моделей (ChatGPT и так далее) за рамками справочной и образовательной информации для генерации кода задания — считается плагиатом, и такое домашнее задание оценивается в 0 баллов.

Всего в сумме можно получить, если правильно решить все задания, 10 баллов.

## Импорт библиотек, установка константных значений

In [ ]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_regression, fetch_california_housing
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
RANDOM_STATE = 42

## Генерация данных

In [ ]:
np.random.seed(RANDOM_STATE)

X, y, _ = make_regression(
    n_samples=100000,              # число объектов
    n_features=9,                  # число признаков
    n_informative=8,               # число информативных признаков
    noise=100,                     # уровень шума в данных
    coef=True,                     # значение True используется при генерации данных
    random_state=RANDOM_STATE
    )

X = pd.DataFrame(data=X, columns=np.arange(0, X.shape[1]))
X[9] = X[6] + X[7] + np.random.random()*0.01

In [ ]:
X.shape, y.shape

((100000, 10), (100000,))

Сгенерировали датасет из 100000 объектов и 10 признаков у каждого.

Обучим модель линейной регрессии:

$$
a(x) = \beta_1 d_{1} + \beta_2 d_{2} + \beta_3 d_{3} + \beta_4 d_{4} + \beta_5 d_{5} + \beta_6 d_{6} + \beta_7 d_{7} + \beta_8 d_{8} + \beta_9 d_{9} + \beta_{10} d_{10} + \beta_0
$$

Которая минимизирует MSE:

$$
Q(a(X), Y) = \sum_i^{100000} (a(x_i) - y_i)^2
$$

## Практика

Реализуем метод градиентного спуска для обучения линейной регрессии.

### Задание 1 (1 балл)


Напишите функцию, вычисляющую значение весов в линейной регрессии по точной (аналитически найденной) формуле:

$$w = (X^TX)^{-1}X^Ty$$

Комментарий: для поиска решения в векторном виде сначала необходимо добавить единичный столбец к матрице $X$.
Это сделано в коде.

In [ ]:
def ols_solution(X, y):
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    # ваш код здесь
    X_transpose = X.T                           #Транспонирую
    XtX = np.dot(X_transpose, X)                #Произведение
    XtX_inv = np.linalg.inv(XtX)                #-1
    Xty = np.dot(X_transpose, y)                #Произведение 2
    w = np.dot(XtX_inv, Xty)                    #finally
    return w

### Задание 2 (1 балл)

Заполните функцию для предсказания модели по формуле
$$a(X)=Xw$$

In [ ]:
def prediction(X, w):
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    # ваш код здесь
    w = np.dot(X, w)

### Задание 3 (1 балл)

Обучите коэффициенты линейной регрессии с помощью библиотеки <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html"> **sklearn** </a>

Отдельно выведите оценку свободного коэффициента  ($\beta_0$ при $d_0 = 1$).

In [ ]:
model = LinearRegression()

# ваш код здесь
model.fit(X, y)

print(model.intercept_)
print(model.coef_)

-1.0699144086277248
[28.10315494  9.26657239 -0.17344479 48.68256004 94.32679765 59.24409726
 22.40916679 13.93763192  0.3641291  36.34679871]


### Задание 4 (1 балл)

Напишите функцию для вычисления среднеквадратичной ошибки

$MSE = \frac{1}{m}||Xw - y||^2_2$.

Здесь квадратичная ошибка записана в матричном виде, т.е. $X$ - матрица объект-признак, $w$ - вектор весов модели.
*  $Xw$ - вектор предсказания модели
*  $y$ - вектор правильных ответов,
и квадратичная ошибка - это квадрат нормы разности вектора предсказания и вектора правильных ответов.

Вычислить норму вектора в Python можно разными способами.  
Воспользуемся готовой функцией из библиотеку numpy - `numpy.linalg.norm`.

In [ ]:
def compute_cost(X, y, theta):
    m = len(y)
    cost = (1 / m) * np.linalg.norm(X@theta - y, 2)**2
    return cost

### Задание 5 (4 балла)

Реализуем градиентный спуск по формуле

$$w_{new} = w_{prev} - \nabla_w Q(w_{prev})$$

Вычислим градиент MSE:
$$\nabla_w Q(w)=\frac2m X^T(Xw-y).$$

Итак, реализуем метод градиентного спуска:

*  первым шагом добавим к матрице `X` единичный столбец - это константный признак, равный 1 на всех объектах.  
Он нужен, чтобы записать предсказание линейной регрессии в виде скалярного произведения и тем самым избавиться от знака суммы:
$a(x)=w_0+w_1x_1+...+w_dx_d=w_1\cdot 1+w_1x_1+...w_dx_d=(w,x)$  
В python скалярное произведение можно записать так: `w@x`

*  затем инициализируем случайным образом вектор весов `params`

*  зададим пустой массив `cost_track`, в который будем записывать ошибку на каждой итерации

*  наконец, в цикле по количеству эпох (итераций) будем обновлять веса по формуле градиентного спуска

In [ ]:
def gradient_descent(X, y, learning_rate, iterations):

    X = np.hstack((np.ones((X.shape[0], 1)), X))
    params = np.random_randint(X.shape[1])

    m = X.shape[0]
    cost_track = []

    for i in range(iterations):
        params = params - learning_rate * (2/m) * (X.T@(X@params - y)) # ваш код здесь
        cost_track.append((1/m) * np.sum((X @ params - y) ** 2)) # ваш код здесь

    return cost_track, params

### Задание 6 (1 балл)

Перепешите метод `gradient_descent` используя одно из трех правил останова

In [ ]:
def gradient_descent(X, y, learning_rate, iterations, epsilon):

    X = np.hstack((np.ones((X.shape[0], 1)), X))
    params = np.random_randint(X.shape[1])

    m = X.shape[0]
    cost_track = []

    for i in range(iterations):
      params = params - learning_rate * (2/m) * (X.T@(X@params - y)) # ваш код здесь
      cost_track.append((1/m) * np.sum((X @ params - y) ** 2)) # ваш код здесь
      if i > 0 and cost_track[i-1] - cost_track[i] < epsilon:
        break

    return cost_track, params

### Задание 7 (1 балл)

- Обучите линейную регрессию тремя методами (по точной формуле, с помощью функции из библиотеки `sklearn` и с помощью GD) на данных для задачи регрессии ($X, y$). Для GD используйте `learning_rate = 0.01, iterations = 10000`, `epsilon = 0.001`.

- С помощью каждого метода сделайте предсказание (на всех данных), вычислите качество предсказания r2 (`from sklearn.metrics import r2_score`). Для получения предсказания использоуйте функцию `predict`.


In [ ]:
# **План**

# 1 - находим веса одним из методов

# 2 - применяем функцию prediction для получения предсказаний с найденными весами

# 3 - вычисляем значение метрики r2